# Prioritize Delay-Sensitive Traffic Using QOS
###### <sup>Use case: Paban Sarma (pasarma@cisco.com); Developed by: Sarah Samuel (sasamuel@cisco.com). </sup>

Quality of Service (QoS) is the technique of prioritizing traffic flows and providing preferential forwarding for higher-priority packets. The fundamental reason for implementing QoS in your network is to provide better service for certain traffic flows.

There are 3 ways to achieve this on a Service-Provider (SP) network using QoS:
* Uniform Mode QoS
* Pipe Mode QoS
* Short-pipe Mode QoS

This notebook demonstrates uniform mode QoS, whereby the packets of delay-sensitive traffic is routed with high-priority, across
the SP network.

### NETWORK TOPOLOGY
This notebook configures a Service Provider network topology, with 2 Provider-Edge (PE) routers and two Provider (P) router. After the base configuration of OSPF and MPLS is up, the play the notebook code-cells to configure QoS on the ingress and egress interfaces of PE and P routers in the network. The QoS policies ensures that delay sensitive traffic such as voice and video are given high priroity.

There are 2 traffic generator ports in this network, which represent 2 Customer Edge (CE) routers. A traffic stream with high priority flows from one traffic generator port to the other to simulate the end-customer's video traffic flow across the SP network.

![Topology](./images/QoS.png)

In order to ensure high priority for video traffic, the customer marks the video traffic packets with certain values in the packet header. In this case, this notebook sends traffic streams with the Differentiated Services Code Point (DSCP) field in the IP header marked for expedited forwarding (EF). Within the PE and P routers in the service-provider network, this notebook will show you how to provide preferential treatment to the marked packets using QoS policies.

## Configuration Steps
* [Access Device Consoles](#Access-Device-Consoles)
* [Configure Base Network](#Configure-Base-Network)
* [Verify Base Network](#Verify-Base-Network)
* [Apply QoS on PE1](#Apply-QoS-on-PE1)
* [Apply QoS on P1](#Apply-QoS-on-P1)
* [Start Traffic](#Start-Traffic)
* [Verify Traffic Prioritization](#Verify-Traffic-Prioritization)


### Access Device Consoles
> Play the following cell to access the SSH console of each device in the topology.

In [ ]:
from lib.xr import *
nodes = {
         'PE1':'', 
         'P1':'',
         'P2':'', 
         'PE2':'', 
         'trex':''
        }

tb = access_device_consoles("lib/tb.yaml", nodes)

### Configure Base Network
>A simple 4 router OSPF MPLS network is the base network. Play the below cell to configure the routers. We have defined the base configuration for each of the routers in the file [xr.py](./lib/xr.py) in the respective configuration strings.

In [ ]:
out = nodes['PE1'].configure(pe1_config_str)
out = nodes['P1'].configure(p1_config_str)
out = nodes['P2'].configure(p2_config_str)
out = nodes['PE2'].configure(pe2_config_str)

### Verify Base Network

> Ensure the basic reachability across the network by checking that the routing protocol (OSPF) and MPLS is up with the below commands.

In [ ]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''
                    show version
                    show ip ospf nei
                    show mpls ldp interface
                    show mpls ldp neigh br
                    show ip route
                    ''')

### Apply QoS on PE1
> The customer edge router has marked the DSCP bits as ef for the high priority video traffic. Play the below cell to configure the class-maps, policy-maps and apply the QoS configuration on the ingress interface of the PE1 router in order to the classify the traffic packets based on their DSCP value in the packet.
> * The ingress policy-map PE-INGRESS ensures that the incoming packets marked with EF for the DSCP field is set to traffic class 7 and marks the mpls exp bits to 7.
> * The egress policy-map VIDEO-PRIORITY ensures that the packets with traffic class 7 are sent out through the high priority output queue of the egress interface. Hence these high-priority traffic packets will be forwarded with minimum delay.
![Topology](./images/PE-policy.png)



In [ ]:
out = nodes['PE1'].configure('''
class-map match-all DSCP-EF
 match dscp ef
 end-class-map
 
class-map match-any Video
 match traffic-class 7
 end-class-map

policy-map PE-INGRESS
 class DSCP-EF
  set traffic-class 7
  set mpls experimental imposition 7
 class class-default
  set traffic-class 0
 end-policy-map
 
policy-map VIDEO-PRIORITY
 class Video
  shape average 2 gbps
  priority level 1
 end-policy-map

interface FourHundredGigE0/0/0/0
 service-policy input PE-INGRESS

interface FourHundredGigE0/0/0/1
 service-policy output VIDEO-PRIORITY
''')

### Apply QoS on P1

> Play the below cell to configure the class-maps, policy-maps and apply the QoS configuration on the P1 router. The QoS policy that we will configure on ingress interface of the P1 router classifies the incoming traffic packets based on their MPLS EXP value. Then the QoS policy that we will configure on the egress interface ensures the traffic packets with the traffic-class 7 gets high priority and assured bandwidth.

![Topology](./images/P-policy.png)


In [ ]:
out = nodes['P1'].configure('''
class-map match-any EXP7
 match mpls experimental topmost 7 
 end-class-map
 
class-map match-any Video
 match traffic-class 7
 end-class-map

policy-map CORE-INGRESS
 class EXP7
  set traffic-class 7
  set mpls experimental imposition 7 
 class class-default
  set traffic-class 0
 end-policy-map

policy-map VIDEO-PRIORITY
 class Video
  shape average 2 gbps
  priority level 1
 class class-default
 end-policy-map

interface FourHundredGigE0/0/0/0
 service-policy input CORE-INGRESS

interface FourHundredGigE0/0/0/1
 service-policy output VIDEO-PRIORITY
''')

### Start Traffic


> TREX is a software traffic generator that runs on Linux. To simulate traffic flow across the 4 router network, TREX software traffic generator ports is connected to routers PE1 and PE2

> A high priority traffic stream is injected into PE1 from TREX with the following details:

>Source IP address: 10.0.0.1

>Destination IP address: 10.1.1.1
![traffic_flow](./images/qos-traffic.png)

In [ ]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)
client1, client2, interact1, interact2 = generate_hipriority_traffic (trex_ipaddress, trex_port)

### Verify Traffic Prioritization
> Play the below cell to check the the outputs of the interface counters and the policy-map on the routers. The interface counters show the number of packets incoming or outgoing from the interface. And the policy-map counters show the number of packets for which the QoS is being applied. In the below show command outputs verify that the packet counters are incrementing in class DSCP-EF in the ingress policy-map and in class Video in the egress policy-map.

In [ ]:
# Check the input and output policies applied on PE1 router.
out = nodes['PE1'].execute('''
show interface FourHundredGigE0/0/0/0 accounting
show interface FourHundredGigE0/0/0/1 accounting
show policy-map interface FourHundredGigE0/0/0/0
show policy-map interface FourHundredGigE0/0/0/1
''')

> In the below show command outputs verify that the packet counters are incrementing in class EXP7 in the ingress policy-map and in class Video in the egress policy-map. This shows that even in subsequent routers in the MPLS network, delay-sensitive traffic can be given high priority.

In [ ]:
out = nodes['P1'].execute('''
show interface FourHundredGigE0/0/0/0 accounting
show interface FourHundredGigE0/0/0/1 accounting
show policy-map interface FourHundredGigE0/0/0/0
show policy-map interface FourHundredGigE0/0/0/1
''')

### Clean-up Network Configurations

In [16]:
   if (n != 'trex')
      out = nodes[n].configure(unconfig_str)
   else
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes[n].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')
      
out = nodes['P1'].configure('''
no policy-map VIDEO-PRIORITY
no policy-map CORE-INGRESS
no class-map match-any Video
no class-map match-any EXP7
''')
out = nodes['PE1'].configure('''
no policy-map VIDEO-PRIORITY
no policy-map PE-INGRESS
no class-map match-any Video
no class-map match-all DSCP-EF
''')

Hope you now have a good idea on using QoS policies to ensure that high-priority traffic is forwarded through the network with minimum delay. If you have any comments or suggestions about this notebook, please reach out to mig-notebooks@cisco.com. We look forward to your feedback.